### 📦 Cell 1 — Import libraries
This cell loads the Python libraries we’ll need:
- **pandas** — for working with tabular data  
- **numpy** — for numeric calculations  
- **matplotlib** — for making plots  
- **statsmodels** — for running statistical tests (ANOVA, post-hoc tests)

Always run this cell first so everything is available.


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests
 

# Load and Clean the Dataset

This cell does three important things:

 ### 1. Loads the Excel file  that contains both WT and ORXKO data 
   
   data = pd.read_excel("data/Tutorial 7 Data.xlsx", sheet_name="WT VS ORXKO DATA")
   
###### Purpose

Import the Excel dataset containing WT (wild-type) and ORXKO (orexin knockout) data into a Pandas DataFrame for analysis.

###### Indepth explanation

pd.read_excel() → A Pandas function that reads spreadsheet data from an Excel file.

"data/Tutorial 7 Data.xlsx" → The file path to the Excel document being imported.

sheet_name="WT VS ORXKO DATA" → Specifies which sheet in the workbook to read. Only that sheet is imported.

The resulting variable data is a DataFrame, a structured table that allows efficient data manipulation and statistical computation.

### 2. Cleans up the group names**  

data["Group"] = data["Group"].replace({
    "Data for WT Mice": "WT",
    "Data for ORX-KO Mice": "ORXKO"})

###### Purpose

Standardize the group labels in the "Group" column by replacing long descriptive strings with short, consistent identifiers used throughout the analysis.

###### Indepth explanation

data["Group"] → Selects the "Group" column from the DataFrame.

.replace({...}) → Substitutes specific text patterns according to the dictionary provided:

"Data for WT Mice" → replaced with "WT"

"Data for ORX-KO Mice" → replaced with "ORXKO"

This ensures uniformity in group names, preventing errors in downstream grouping, statistical analysis, and plotting.

### 3. Print out data

print("✅ Data loaded successfully!")
print("Groups found:")
display(data)

###### Purpose

Provide a quick confirmation that the Excel data has been successfully loaded and display the dataset for visual inspection.

###### indepth explanation

print("✅ Data loaded successfully!") → Prints a success message to indicate that the file has been read into Python without any errors. The green checkmark emoji (✅) makes the output visually easy to spot in a long notebook run.

print("Groups found:") → Acts as a heading to introduce the following dataset display, reminding the reader that the upcoming DataFrame output should contain both WT and ORXKO group entries.

display(data) → Uses Jupyter Notebook’s built-in display function (as opposed to print()) to neatly render the DataFrame in a tabular format.
This allows you to visually verify that:

The file was read correctly.

The "Group" column contains standardized group names (WT and ORXKO).

The data structure looks as expected before performing statistical analysis.

✅ **In short:** this cell loads the Excel file, cleans the labels, and checks that the dataset is ready to use.


In [ ]:
# Load the Excel file that contains both WT and ORXKO data
data = pd.read_excel("data/Tutorial 7 Data.xlsx", sheet_name="WT VS ORXKO DATA")

# Clean up the group column
data["Group"] = data["Group"].replace({
    "Data for WT Mice": "WT",
    "Data for ORX-KO Mice": "ORXKO"
})

print("✅ Data loaded successfully!")
print("Groups found:")

display(data)

# Compute Descriptive Statistics (Means and Standard Deviations)

Before we perform any statistical tests, it’s important to summarize our data by calculating:

The average (mean) Total % of each sleep state for both WT and ORXKO mice.

The standard deviation (SD), which tells us how much individual mice vary around that average. 
data → our full dataset


### 1. Define Variables 

(data)VARIABLE = ["VARIABLEAW", "VARIABLEQW", "VARIABLENREM", "VARIABLEREM"]

###### Purpose

- This line creates a list of column names that represent the percent of time spent in each sleep state.

###### More indepth explanation of code above ^

- VARIABLE = [...] → defines a Python list that groups related column names together.

- Each string inside the list (like "VARIABLEAW") corresponds to a column name in your dataset.

### 2. Compute Group Statistics

wt_means = data[data["Group"]=="WT"][VARIABLE].mean()
orxko_means = data[data["Group"]=="ORXKO"][VARIABLE].mean()

###### Purpose

- These lines calculate the average percent time in each sleep state for WT and ORXKO mice.

###### More indepth explanation of code above ^

- data["Group"]=="WT" → creates a filter showing which rows belong to WT mice

- data[data["Group"]=="WT"] → applies that filter, keeping WT rows only

- [VARIABLE] → selects only the columns with  VARIABLE

- .mean() → computes the average for each sleep state

- The same structure is used for .std() to compute standard deviations.


### 3. Display the Results

display(pd.DataFrame({
    "WT Mean": wt_means,
    "ORXKO Mean": orxko_means,
    "WT SD": wt_std,
    "ORXKO SD": orxko_std
}))

###### Purpose 

- This combines the results into a table showing the average and variability for each sleep state across both groups.

###### More indepth expplanation of code above ^ 

- pd.DataFrame({...}) → creates a new table using the pandas library.

- Each label (e.g., "WT Mean", "ORXKO SD") becomes a column header in the table.

- The variables (wt_means, orxko_means, etc.) supply the actual values for each column.

- display(...) → neatly prints the table so it’s easy to view inside a Jupyter notebook.



In [ ]:
# ---- Define variables ----
VARIABLE = ["VARIABLEAW", "VARIABLEQW", "VARIABLENREM", "VARIABLEREM"]

# ---- Compute group statistics ----
wt_means = data[data["Group"]=="WT"][VARIABLE].mean()
orxko_means = data[data["Group"]=="ORXKO"][VARIABLE].mean()

wt_std = data[data["Group"]=="WT"][VARIABLE].std()
orxko_std = data[data["Group"]=="ORXKO"][VARIABLE].std()

print("✅ Descriptive statistics computed!\n")
display(pd.DataFrame({"WT Mean": wt_means, "ORXKO Mean": orxko_means,
                      "WT SD": wt_std, "ORXKO SD": orxko_std}))



#  Preparing Data and Running a Two-Way ANOVA

In this step, we reformat our dataset and perform a two-way ANOVA (Analysis of Variance) to test whether group (WT vs ORXKO), sleep state (AW, QW, NREM, REM), or their interaction affects whatever variable we are testing (Percent of time, bouts, duration ect).

considers all groups and all interactions between paramters testing 

anova tells ytpu there are differences but not where they are 

Cant exactly say that its just the gentic backgrtound that leads to difference of interaction group is positive 

emphasis on group - group diff

### 1. Reshape data to long format

stats_data = data.melt(
    id_vars=["Mouse", "Group"],
    value_vars=VARIABLE,
    var_name="State",
    value_name="VARIABLE"
)
stats_data["State"] = stats_data["State"].str.replace("VARIABLE", "")

###### Purpose

- Turn wide columns (e.g., VARIABLEAW, …_QW, etc.) into a single column VARIABLE with a companion label column State. This is the format that ANOVA functions expect.

###### Indepth explanation of code above

- data.melt() → The pandas function that converts from wide → long format.

- id_vars=["Mouse", "Group"] → Keeps these columns fixed for each row after melting (so you know which mouse and group each value belongs to).

- value_vars=VARIABLE → Targets all columns that start with "VARIABLE" (like VARIABLEAW, VARIABLEQW, etc.).

- var_name="State" → Creates a new column called "State" that stores what column the value came from — e.g. "VARIABLEAW", "VARIABLEREM", etc.

- value_name="VARIABLE" → The name of the new column that will contain the numeric values (e.g. 40, 10, 35, etc.).

| Group       | Mouse     | State             | VARIABLE |
| ----------- | --------- | ----------------- | ------------- |
| Data for WT | 503 WT    | VARIABLEAW   | 20.93         |
| Data for WT | 503 WT    | VARIABLEQW   | 5.88          |
| Data for WT | 503 WT    | VARIABLENREM | 61.34         |
| Data for WT | 503 WT    | VARIABLEREM  | 11.85         |
| Data for WT | 504 WT    | VARIABLEAW   | 32.37         |
| Data for WT | 504 WT    | VARIABLEQW   | 7.90          |
| Data for WT | 504 WT    | VARIABLENREM | 54.14         |
| Data for WT | 504 WT    | VARIABLEREM  | 5.58          |
| Data for OR | 313 ORXKC | VARIABLEAW   | 36.52         |
| Data for OR | 313 ORXKC | VARIABLEQW   | 7.71          |
| Data for OR | 313 ORXKC | VARIABLENREM | 48.10         |
| Data for OR | 313 ORXKC | VARIABLEREM  | 7.65          |


### 2. Two-way Anova

model = ols("VARIABLE ~ C(Group) * C(State)", data=stats_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

###### Purpose 
- Fit an Ordinary Least Squares (OLS) model using ols() to examine how VARIABLE varies across different Groups and States, and whether there’s an interaction between them.
- Pass the fitted model into anova_lm() to generate a Type II ANOVA table, which calculates F-values and p-values for each main effect and their interaction.

###### Indepth explanation of code above
- ols() → Fits an Ordinary Least Squares regression model using a formula interface.

- "VARIABLE ~ C(Group) * C(State)" → Defines the model formula where VARIABLE is the dependent variable, and both Group and State are categorical factors (the C() ensures they’re treated as such). The * automatically expands to include both main effects (Group, State) and their interaction (Group:State).

- data=stats_data → Specifies the DataFrame that contains all the variables used in the formula.

- .fit() → Fits (executes) the model and stores the results (like coefficients, residuals, etc.) in the variable model.

- sm.stats.anova_lm() → Runs an ANOVA (Analysis of Variance) on the fitted model to summarize how much variance each factor explains.

- typ=2 → Performs a Type II ANOVA, which tests each main effect while considering the other, and includes the interaction term.

- anova_table → Stores the resulting ANOVA table with key statistics: Sum of Squares (SS), Degrees of Freedom (df), F-values, and p-values (PR(>F)).

### 3. Add significance columns

ALPHA = 0.05  # change if you want a different threshold

def p_to_stars(p):
    if pd.isna(p):   return ""
    if p < 0.001:    return "***"
    if p < 0.01:     return "**"
    if p < 0.05:     return "*"
    return "ns"

anova_table["Significant?"] = np.where(anova_table["PR(>F)"] < ALPHA, "Yes", "No")
anova_table["Stars"] = anova_table["PR(>F)"].apply(p_to_stars)

###### Purpose

- Define a significance threshold and helper function to make the ANOVA results easier to interpret.

- Add two new columns to the ANOVA table: one that marks if the p-value is significant (Yes/No), and another that converts the p-value into star notation (*, **, ***, or ns).

###### Indepth explanation of code above 

- ALPHA = 0.05 → Sets the significance cutoff. Any p-value below 0.05 is considered statistically significant.

- def p_to_stars(p): → Defines a helper function that converts a numeric p-value into a string of significance stars.

- if pd.isna(p): return "" → Returns a blank value if the p-value is missing (NaN).

- if p < 0.05: return "*" → Assigns one star for mild significance. More stars more signifcance
- return "ns" → Marks results as not significant if the p-value is ≥ 0.05.

- anova_table["Significant?"] = np.where(anova_table["PR(>F)"] < ALPHA, "Yes", "No") → Creates a new column called "Significant?", labeling each test as "Yes" if its p-value is below the alpha threshold, otherwise "No".

- anova_table["Stars"] = anova_table["PR(>F)"].apply(p_to_stars) → Applies the p_to_stars() function to each p-value in the "PR(>F)" column, creating a new "Stars" column with the corresponding significance symbol.




In [ ]:
# ---- Reshape data to long format for ANOVA ----
stats_data = data.melt(id_vars=["Mouse", "Group"],
                    value_vars=VARIABLE,
                    var_name="State",
                    value_name="VARIABLE")
stats_data["State"] = stats_data["State"].str.replace("VARIABLE", "")

# ---- Two-way ANOVA ----
model = ols("VARIABLE~ C(Group) * C(State)", data=stats_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# ---- Add significance columns ----
ALPHA = 0.05  # change if you want a different threshold

def p_to_stars(p):
    if pd.isna(p):      return ""
    if p < 0.001:       return "***"
    if p < 0.01:        return "**"
    if p < 0.05:        return "*"
    return "ns"

anova_table["Significant?"] = np.where(anova_table["PR(>F)"] < ALPHA, "Yes", "No")
anova_table["Stars"] = anova_table["PR(>F)"].apply(p_to_stars)

# ---- Format F and PR(>F) columns ----
anova_table["F"] = anova_table["F"].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else "")
anova_table["PR(>F)"] = anova_table["PR(>F)"].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "")

print("📊 Two-way ANOVA results:\n")
display(anova_table)


# Bonferroni Post-hoc Test


### 1. Contaiers 

results = []
pvals = []

###### Purpose
Creat empty Python list to store results from each state and to collect raw p-values so we can correct them later. 

###### Indepth explanation 
results = [] → Creates an empty list to store full test results for each sleep state (state name, t-statistic, p-value, etc.).

pvals = [] → Creates an empty list to collect only raw p-values, so they can all be corrected later in one call to multipletests().

### 2. Loop through each sleep state 

for state in stats_data["State"].unique():
    subset = stats_data[stats_data["State"] == state]


###### Purpose:
Iterate through each unique sleep state (AW, QW, NREM, REM) and isolate the rows belonging to that specific state so that later calculations (like t-tests) are performed separately for each condition.

###### In-depth explanation 

for state in stats_data["State"].unique(): → Loops once for every unique value in the "State" column of the DataFrame. Example: if your dataset contains 4 states (AW, QW, NREM, REM), the loop runs exactly 4 times.

subset = stats_data[stats_data["State"] == state] →  keep only the rows where the "State" column equals the current loop value (state)

The result (subset) is a smaller DataFrame that contains data for one state only — for example, all “NREM” rows.


### 3. Extract group vectors (WT vs ORXKO) for this state

wt = subset[subset["Group"] == "WT"]["VARIABLE"]
orxko = subset[subset["Group"] == "ORXKO"]["VARIABLE"]
 
###### Purpose 

From the filtered state-specific DataFrame (subset), pull out the percentage-of-time data (VARIABLE) for each genotype (WT and ORXKO). These two numeric vectors will later be compared statistically.

###### In-depth explanation

subset["Group"] == "WT" → Creates a boolean mask (True/False values) marking which rows belong to the WT group.

subset[subset["Group"] == "WT"] → Applies that mask to keep only WT rows in the subset.

["VARIABLE"] → Selects just the column containing the dependent variable (% time spent in that state).
The result is a Pandas Series (a 1D array) of WT values for that sleep state.

The same logic applies to the ORXKO line — but it filters rows where "Group" == "ORXKO".
You end up with two arrays, wt and orxko, representing the data distributions for each group in that specific state.


### 4. Run independent t-test for WT vs ORXKO within this state
 
 t_stat, p_val = ttest_ind(wt, orxko, equal_var=True)
    results.append({"State": state, "t_stat": t_stat, "p_raw": p_val})
    pvals.append(p_val)

###### Purpose:
Compute a two-sample t-test comparing groups within the current state, and save both the t statistic and the raw p-value.

###### In-depth explanation: 

t_stat, p_val = ttest_ind(wt, orxko, equal_var=True) → Performs an independent two-sample t-test assuming equal variance between WT and ORXKO.

t_stat → The standardized difference between the two means.

p_val → The probability of observing a t this extreme if there were no true group difference.

results.append({"State": state, "t_stat": t_stat, "p_raw": p_val}) → Adds one dictionary entry (state name + test results) to the results list.

pvals.append(p_val) → Appends the raw p-value to a separate list for later correction.


### 5. Apply Bonferroni correction to the 4 p-values

reject, p_adj, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni')

###### Purpose:

Control the familywise Type I error across the 4 planned comparisons by applying Bonferroni correction.

######  indepth explanation 

reject, p_adj, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni') → Runs the Bonferroni multiple-comparison correction.

pvals → The list of 4 raw p-values, one per state.

alpha=0.05 → Overall significance level for the full set of tests.

method='bonferroni' → Multiplies each p-value by the number of tests (4) and caps them at 1.

reject → Boolean list: True if the corrected p-value < 0.05.

p_adj → Bonferroni-adjusted p-values.

_ , _ → Unused outputs (critical thresholds).



### 6. Add corected values to the results 

for i in range(len(results)):
    results[i]["p_adj"] = p_adj[i]
    results[i]["reject"] = reject[i]
    
###### Purpose:
 Merge the adjusted p-values and significance flags back into each state's result row

###### In-depth explanation:

for i in range(len(results)): → Loops over each state’s entry in the results list.

results[i]["p_adj"] = p_adj[i] → Adds the adjusted p-value to that state’s dictionary.

results[i]["reject"] = reject[i] → Adds a flag (True/False) indicating whether the state difference is significant after correction.

### 7. Display the results 

results_df = pd.DataFrame(results)
display(results_df)

Convert the list of per-state dictionaries into a tidy table for inspection/printing.

###### Purpose 

Convert the list of per-state dictionaries into a tidy table for inspection/printing.

###### Indepth explanation

results_df = pd.DataFrame(results) → Converts the list of dictionaries into a Pandas DataFrame for easy viewing and plotting.

display(results_df) → Outputs the table of all state-level statistics, including both raw and Bonferroni-corrected p-values.





In [ ]:


from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import pandas as pd

# Containers
results = []
pvals = []

# Loop through each sleep state
for state in stats_data["State"].unique():
    subset = stats_data[stats_data["State"] == state]
    
# Extract group vectors (WT vs ORXKO) for this state
    wt = subset[subset["Group"] == "WT"]["VARIABLE"]
    orxko = subset[subset["Group"] == "ORXKO"]["VARIABLE"]
    
    # Run independent t-test for WT vs ORXKO within this state
    t_stat, p_val = ttest_ind(wt, orxko, equal_var=True)
    results.append({"State": state, "t_stat": t_stat, "p_raw": p_val})
    pvals.append(p_val)

# Apply Bonferroni correction to the 4 p-values
reject, p_adj, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni')

# Add corrected values to the results
for i in range(len(results)):
    results[i]["p_adj"] = p_adj[i]
    results[i]["reject"] = reject[i]

# Display the results
results_df = pd.DataFrame(results)
display(results_df)



#### Plot setup 

x = np.arange(len(VARIABLE))
width = 0.35
fig, ax = plt.subplots(figsize=(8,5))

###### Purpose:

Initialize the x-axis positions, bar spacing, and figure size to prepare the layout for plotting.

###### In-depth explanation:

np.arange(len(VARIABLE)) → Generates evenly spaced positions [0, 1, 2, 3] for each state on the x-axis.

width = 0.35 → Sets the horizontal width of each bar, allowing side-by-side WT and ORXKO bars.

plt.subplots(figsize=(8,5)) → Creates the figure (fig) and axes (ax) objects for plotting. The figure will be 8×5 inches.

#### Plot bars using stored means and SDs

bars1 = ax.bar(x - width/2, wt_means, width, yerr=wt_std,
               capsize=5, label="WT", color="purple", alpha=0.8)
bars2 = ax.bar(x + width/2, orxko_means, width, yerr=orxko_std,
               capsize=5, label="ORXKO", color="pink", alpha=0.8)
###### Purpose:

Plot the group mean percentage of time spent in each sleep state for WT and ORXKO mice with error bars showing standard deviation.

###### In-depth explanation:

x - width/2 and x + width/2 → Offset the WT and ORXKO bars so they appear side-by-side instead of overlapping.

yerr= → Plots standard deviation error bars above each bar.

capsize=5 → Adds small end caps to each error bar for readability.

color, alpha, label → Define bar color, transparency, and legend label for each group.

The resulting objects (bars1, bars2) store the bars for later reference if needed.

#### Scatter individual data points

for i, col in enumerate(VARIABLE):
    y_wt = data[data["Group"] == "WT"][col]
    x_wt = np.random.normal(i - width/2, 0.04, size=len(y_wt))
    plt.scatter(x_wt, y_wt, color="black", s=25, alpha=0.7)

    y_orx = data[data["Group"] == "ORXKO"][col]
    x_orx = np.random.normal(i + width/2, 0.04, size=len(y_orx))
    plt.scatter(x_orx, y_orx, color="black", s=25, alpha=0.7)
    
###### Purpose:

Overlay each mouse’s raw data as jittered points to show individual variation and distribution within each group.

###### in-depth explanation 

for i, col in enumerate(VARIABLE) → Loops through each state index (i) and variable name (col).

data[data["Group"] == "WT"][col] → Extracts all WT mouse values for the current sleep state.

np.random.normal(i - width/2, 0.04, size=len(y_wt)) → Adds small horizontal “jitter” (±0.04) so dots don’t overlap perfectly.

plt.scatter() → Plots individual points as black semi-transparent circles (alpha=0.7, s=25).

The same steps repeat for ORXKO, centered at i + width/2 to align with their bars.

#### Add significance stars using saved Bonferroni p-values

def p_to_stars(p):
    if p < 0.001: return "***"
    elif p < 0.01: return "**"
    elif p < 0.05: return "*"
    else: return "ns"
###### Purpose:
 
Convert numeric Bonferroni-adjusted p-values into significance labels (***,**,*, or “ns”).

###### Indepth expanation

def p_to_stars(p): → Defines a simple helper function that takes a single numeric argument (p, the p-value).

if p < 0.001: → return "***" → If the p-value is smaller than 0.001, it indicates a very strong level of statistical significance, so three stars are returned.

elif p < 0.01: → return "**" → For p-values below 0.01 but above 0.001, two stars are returned, representing moderate significance.

elif p < 0.05: → return "*" → For p-values below 0.05 but above 0.01, one star is returned, indicating the typical 5% significance threshold.

else: return "ns" → “ns” stands for not significant, returned when the p-value is greater than or equal to 0.05.


#### Extract Bonferroni-corrected p-values per state name

state_labels = ["AW", "QW", "NREM", "REM"]
pvals_bonf = [results_df.loc[results_df["State"] == state, "p_adj"].values[0] for state in state_labels]

###### Purpose:

Retrieve the Bonferroni-corrected p-values (p_adj) for each sleep state from the results_df DataFrame in the same order as the plotted states.

###### In-depth explanation:

state_labels = ["AW", "QW", "NREM", "REM"] → Creates an ordered list of the four sleep states, ensuring consistency between your bar plot’s x-axis and the corresponding p-values retrieved for annotation.

[ ... for state in state_labels] → This is a list comprehension that loops through each state name in state_labels, performing a lookup in the results table for that specific state.

results_df.loc[results_df["State"] == state, "p_adj"] → Uses .loc[] to filter the results_df DataFrame where the "State" column matches the current state being looped over. Selects the "p_adj" column (the Bonferroni-corrected p-value) for that filtered row.

.values[0] → Extracts the single numeric p-value from the resulting array. Since each state appears only once in the results table, [0] ensures we grab just that one value.

The final output, pvals_bonf, is a list of four p-values (one per state: AW, QW, NREM, REM), ready to be passed into the annotation loop to determine how many significance stars to display on the graph.


#### Annotate significance stars above each bar pair

for i, p in enumerate(pvals_bonf):
    stars = p_to_stars(p)
    y = max(wt_means[i], orxko_means[i]) + 6
    ax.text(i, y, stars, ha="center", va="bottom", fontsize=12, fontweight="bold")

###### Purpose:

Place the appropriate significance symbol (e.g., ***, **, *, or ns) above each pair of bars (WT vs ORXKO) for each sleep state based on the corrected p-values.

###### In-depth explanation:
 
for i, p in enumerate(pvals_bonf): → Loops through each Bonferroni-corrected p-value in pvals_bonf, while keeping track of the index i that corresponds to each sleep state’s bar position.

stars = p_to_stars(p) → Passes the current p-value into the p_to_stars() function (defined earlier) to convert the numeric p-value into a text-based significance marker.

y = max(wt_means[i], orxko_means[i]) + 6 → Calculates the y-position (height) at which to place the text annotation.

max(wt_means[i], orxko_means[i]) → Finds which group (WT or ORXKO) has the higher mean value for that state.

+ 6 → Adds a small vertical offset so the stars appear slightly above the taller bar, not touching it.

ax.text(i, y, stars, ha="center", va="bottom", fontsize=12, fontweight="bold") →
Draws the significance label directly onto the plot at the correct x and y coordinates.

i → x-position corresponds to the sleep state’s index on the x-axis.

y → y-position is above the tallest bar.

ha="center" → Centers the text horizontally over the bar pair.

va="bottom" → Aligns text from the bottom, ensuring it sits neatly above the bar tops.

fontsize=12, fontweight="bold" → Makes the significance symbols large and bold for clarity in presentations and publications.

#### Aesthetics 

ax.set_xticks(x)
ax.set_xticklabels(state_labels)
ax.set_ylabel("Number of Bouts (Mean ± SD)")
ax.set_title("Number of Bouts Per State\nWT vs ORXKO Mice")
ax.legend(title="Group")
ax.spines[['top', 'right']].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

###### Purpose:

Finalize and polish the visual style of the plot by adjusting labels, legends, gridlines, and layout for clarity and professional presentation.

###### Indepth explanation

ax.set_xticks(x) → Specifies the exact x-axis positions where tick marks should appear. These correspond to the bar locations for each sleep state (AW, QW, NREM, REM).

ax.set_xticklabels(state_labels) → Assigns human-readable labels to each tick mark using the state_labels list, ensuring each x-axis tick corresponds to a specific sleep state.

ax.set_ylabel("Number of Bouts (Mean ± SD)") →
Adds a descriptive y-axis label explaining what the plotted values represent — the mean percentage of time spent in each sleep state with its standard deviation.

ax.set_title("Number of Bouts Per State\nWT vs ORXKO Mice") →
Adds a plot title summarizing the content. The \n creates a line break, neatly separating the main title and subtitle.

ax.legend(title="Group") → Displays a legend identifying which color represents each group (WT or ORXKO).
The title argument labels the legend box for additional clarity.

ax.spines[['top', 'right']].set_visible(False) → Removes the top and right borders of the graph for a cleaner, modern appearance that emphasizes the data rather than the frame.

ax.grid(axis='y', linestyle='--', alpha=0.4) → Adds faint horizontal grid lines to improve readability of the bar heights.

axis='y' → Applies the grid only to the y-axis.

linestyle='--' → Dashed lines maintain a subtle visual guide.

alpha=0.4 → Makes the grid semi-transparent to avoid distraction.

plt.tight_layout() → Automatically adjusts spacing between elements (axes, labels, title) so that nothing overlaps or gets cut off.

plt.show() → Renders and displays the final plot output in the notebook or window.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# ---- Plot setup ----
x = np.arange(len(VARIABLE))
width = 0.35
fig, ax = plt.subplots(figsize=(8,5))

# ---- Plot bars using stored means and SDs ----
bars1 = ax.bar(x - width/2, wt_means, width, yerr=wt_std,
               capsize=5, label="WT", color="purple", alpha=0.8)
bars2 = ax.bar(x + width/2, orxko_means, width, yerr=orxko_std,
               capsize=5, label="ORXKO", color="pink", alpha=0.8)

# ---- Scatter individual data points ----
for i, col in enumerate(VARIABLE):
    y_wt = data[data["Group"] == "WT"][col]
    x_wt = np.random.normal(i - width/2, 0.04, size=len(y_wt))
    plt.scatter(x_wt, y_wt, color="black", s=25, alpha=0.7)

    y_orx = data[data["Group"] == "ORXKO"][col]
    x_orx = np.random.normal(i + width/2, 0.04, size=len(y_orx))
    plt.scatter(x_orx, y_orx, color="black", s=25, alpha=0.7)

# ---- Add significance stars using SAVED Bonferroni p-values ----
def p_to_stars(p):
    if p < 0.001: return "***"
    elif p < 0.01: return "**"
    elif p < 0.05: return "*"
    else: return "ns"

# ---- Extract Bonferroni-corrected p-values per state name ----
state_labels = ["AW", "QW", "NREM", "REM"]
pvals_bonf = [results_df.loc[results_df["State"] == state, "p_adj"].values[0]
              for state in state_labels]

for i, p in enumerate(pvals_bonf):
    stars = p_to_stars(p)
    y = max(wt_means[i], orxko_means[i]) + 6
    ax.text(i, y, stars, ha="center", va="bottom", fontsize=12, fontweight="bold")

# ---- Aesthetics ----
ax.set_xticks(x)
ax.set_xticklabels(state_labels)
ax.set_ylabel("Number of Bouts (Mean ± SD)")
ax.set_title("Number of Bouts Per State\nWT vs ORXKO Mice")
ax.legend(title="Group")
ax.spines[['top', 'right']].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()
